![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 2: Basic Chisel : Control Flow

#### Written by Chick Markley, Stevo Bailey, Edward Wang, and Richard Lin
[chick@berkeley.edu](mailto:chick@berkeley.edu),
[stevo@berkeley.edu](mailto:stevo@berkeley.edu),
[edwardw@berkeley.edu](mailto:edwardw@berkeley.edu), [richard.lin@berkeley.edu](mailto:edwardw@berkeley.edu)

## Introduction

Up until now there has been a strong correspondence between software and hardware in Chisel.
In control flow there will be a greater divergence between the way we think about the two.

## Table of Contents

1. [Last Connect Semantics](#last-connect)
1. [when/elsewhen/other](#when)
1. [Wire vs val and var](#wire)
1. [Exercise](#exercise)
1. [Optional Exercise ](#optional-exercise)


## Setup

First we repeat the necessary imports to run Chisel.

In [6]:
import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

// A wrapper function that provides a easy way to get the generated Verilog code for a Chisel module.
// Don't worry about the implementation of this.
def getVerilog[T <: Module](gen: => T): String = {
  chisel3.Driver.execute(Array[String](), {() => gen}) match {
    case ChiselExecutionSuccess(_, _, Some(firrtl.FirrtlExecutionSuccess(_, verilog))) => verilog
  }
}

import $ivy.$                                                 

import $ivy.$                                                          

import $ivy.$                                                

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

// A wrapper function that provides a easy way to get the generated Verilog code for a Chisel module.
// Don't worry about the implementation of this.

defined function getVerilog

## Last Connect Semantics<a name="last-connect"></a>
 As seen earlier Chisel allows you to connect components using the ```:=``` operator.
 For various reasons it is possible to issue multiple connect statements to the same component.
 When this happens the last statement wins.  

In [7]:
class LastConnect extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := 1.U
  io.out := 2.U
  io.out := 3.U
  io.out := 4.U
}

// Chisel Code: Declare a new tester for Simple modules
class LastConnectTester(c: LastConnect) extends PeekPokeTester(c) {
  expect(c.io.out, 4)  // Assert that the output correctly has 0
}

// Scala Code: Calling Driver to instantiate Simple, SimpleTester, and execute the test
// Don't worry about understanding this code, it is very complicated Scala
val works = Driver(() => new LastConnect) {
  c => new LastConnectTester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 5.0 ms
Total FIRRTL Compile Time: 4.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507329888292
test cmd6WrapperHelperLastConnect Success: 1 tests passed in 5 cycles taking 0.001490 seconds
[info] [0.001] RAN 0 CYCLES PASSED
SUCCESS!!


defined class LastConnect
defined class LastConnectTester
works: Boolean = true

## when, elsewhen, and otherwise<a name="when"></a> 
Chisel's primary implementation of conditional logic is the when, elsewhen, and otherwise constructs. 
This generally looks like
```scala
when(someBooleanCondition) {
  // things to do when true
}.elsewhen(someOtherBooleanCondition) {
  // things to do on this condition
}.otherwise {
  // things to do if none of th boolean conditions are true
}
```
They must must appear in the above order, though either of the latter may be omitted.
There can be as many elsewhen clauses as desired.
Any section that is true terminates the construct. (VERIFY THIS)
Actions taken in the bodies of the the three can be complex blocks and may contain nested
*when* and allies.
>**Unlike** scala ```if``` values are not returned by the blocks associated with when/elsewhen/otherwise.
One cannot say
```scala
val result = when(squareIt) { x * x }.otherwise { x }
```
This will **not** work. We will discuss the solution to this in the *Wires* section.


In [8]:
/** Max3 returns the max of it's 3 arguments
  */
class Max3 extends Module {
  val io = IO(new Bundle {
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out = Output(UInt(16.W))
  })
    
  when(io.in1 > io.in2 && io.in1 > io.in3) {
    io.out := io.in1  
  }.elsewhen(io.in2 > io.in1 && io.in2 > io.in3) {
    io.out := io.in2 
  }.otherwise {
    io.out := io.in3
  }
}

// verify that the max of the three inputs is correct
class Max3Tester(c: Max3) extends PeekPokeTester(c) {
  poke(c.io.in1, 6)
  poke(c.io.in2, 4)  
  poke(c.io.in3, 2)  
  expect(c.io.out, 6)  // input 1 should be biggest
  poke(c.io.in2, 7)  
  expect(c.io.out, 7)  // now input 2 is
  poke(c.io.in3, 11)  
  expect(c.io.out, 11) // and now input 3
  poke(c.io.in3, 3)  
  expect(c.io.out, 7)  // show that decreasing an input works as well
}

// Scala Code: Calling Driver to instantiate Simple, SimpleTester, and execute the test
// Don't worry about understanding this code, it is very complicated Scala
val works = Driver(() => new Max3) {
  c => new Max3Tester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

[info] [0.000] Elaborating design...
[info] [0.006] Done elaborating.
Total FIRRTL Compile Time: 17.5 ms
Total FIRRTL Compile Time: 14.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507329889074
test cmd7WrapperHelperMax3 Success: 4 tests passed in 5 cycles taking 0.007078 seconds
[info] [0.004] RAN 0 CYCLES PASSED
SUCCESS!!


defined class Max3
defined class Max3Tester
works: Boolean = true

## The Wire construct<a name="wire"></a>
Let's return to the note above describing the limitation that **when** does not return a value.
The Chisel **Wire** construct is one of the ways around this.
**Wire** defines a circuit component that can appear on the right hand side or left hand side of
a connect **:=** operator.


To illustrate this let's make a small combinational sorter that sorts its four numeric inputs into
its four numeric outputs. To make things clearer, consider the following graph.  Data follows the red lines
at each step when the left value is less than the right, and follows the black lines which swap the values when the left is greater than the right.
![Sort4](images/Sort4.svg)
The diagram shows a series of cells whose names begin with *row*, we will use **Wire**s to construct these as pwhere results of a particular copy or swap are placed. 

In [9]:
/** Sort4 sorts its 4 inputs to its 4 outputs */
class Sort4 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(io.in0 < io.in1) {
    row10 := io.in0            // preserve first two elements
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // swap first two elements
    row11 := io.in0
  }

  when(io.in2 < io.in3) {
    row12 := io.in2            // preserve last two elements
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // swap last two elements
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(row11 < row12) {
    row21 := row11            // preserve middle 2 elements
    row22 := row12
  }.otherwise {
    row21 := row12            // swap middle two elements
    row22 := row11
  }

  when(row10 < row21) {
    io.out0 := row10            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row10
  }

  when(row10 < row21) {
    io.out0 := row10            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row10
  }

  when(row22 < row13) {
    io.out2 := row22            // preserve first two elements
    io.out3 := row13
  }.otherwise {
    io.out2 := row13            // swap first two elements
    io.out3 := row22
  }
}

// verify that the max of the three inputs is correct
class Sort4Tester(c: Sort4) extends PeekPokeTester(c) {
  poke(c.io.in0, 3)
  poke(c.io.in1, 6)
  poke(c.io.in2, 9)
  poke(c.io.in3, 12)
  expect(c.io.out0, 3)
  expect(c.io.out1, 6)
  expect(c.io.out2, 9)
  expect(c.io.out3, 12)

  poke(c.io.in0, 13)
  poke(c.io.in1, 4)
  poke(c.io.in2, 6)
  poke(c.io.in3, 1)
  expect(c.io.out0, 1)
  expect(c.io.out1, 4)
  expect(c.io.out2, 6)
  expect(c.io.out3, 13)
}

// Scala Code: Calling Driver to instantiate Simple, SimpleTester, and execute the test
// Don't worry about understanding this code, it is very complicated Scala

val works = iotesters.Driver(() => new Sort4) {
c => new Sort4Tester(c)
}
assert(works) // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


[info] [0.000] Elaborating design...
[info] [0.010] Done elaborating.
Total FIRRTL Compile Time: 44.8 ms
Total FIRRTL Compile Time: 31.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1507329889847
test cmd8WrapperHelperSort4 Success: 8 tests passed in 5 cycles taking 0.011189 seconds
[info] [0.007] RAN 0 CYCLES PASSED
SUCCESS!!


defined class Sort4
defined class Sort4Tester
works: Boolean = true

## Control Flow Exercise
Coming Soon